In [1]:
from functions import *
import pandas as pd

In [2]:
#file reading
file_list = ['application_test.csv','credit_card_balance.csv',
             'installments_payments.csv','POS_CASH_balance.csv','previous_application.csv']
app_test, ccb, ip, pcb, pa = one_fell_reader(file_list)

In [3]:
#merging
ccb.drop('SK_ID_CURR',axis=1,inplace=True)
pcb = pcb.drop_duplicates('SK_ID_PREV')
pcb.drop('SK_ID_CURR',axis=1,inplace=True)
ip = ip.groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).agg('mean')
ip = ip.drop_duplicates('SK_ID_PREV')
pa = pa.drop_duplicates('SK_ID_CURR')
pa = pa.drop_duplicates('SK_ID_PREV')
ip_ccb = ip.merge(ccb, on='SK_ID_PREV', how='left')
ip_ccb_pcb = ip_ccb.merge(pcb,on='SK_ID_PREV',how='left')
ip_ccb_pcb.drop('SK_ID_CURR',axis=1, inplace=True)
pa_ip_ccb_pcb = pa.merge(ip_ccb_pcb, on='SK_ID_PREV', how='left')
pa_ip_ccb_pcb.drop('SK_ID_PREV',axis=1,inplace=True)
pa_ip_ccb_pcb = pa_ip_ccb_pcb.drop_duplicates('SK_ID_CURR')
super_df = app_test.merge(pa_ip_ccb_pcb, on='SK_ID_CURR', how='left') #final merge

In [4]:
#dropping all columns with 50% or more null values
super_df = one_fell_dropper(super_df)

In [5]:
#imputing missing data
super_df = one_fell_imputer(super_df)

In [6]:
#encoding all categorical data
super_df = one_fell_encoder(super_df,'TARGET','binary','encoders.pickle',test=True)

In [7]:
#creating the xgbclassifier model
model = one_fell_modeler(test=super_df)